In [1]:
include("chess.jl")

moves (generic function with 1 method)

In [6]:
# initialize a position
pos = Board(["King" => (3,3), "Rook" => (3,2)], ["King" => (3,1)])
moves(pos, 1)


13-element Array{(String,(Int64,Int64)),1}:
 ("Rook",(4,2))
 ("Rook",(5,2))
 ("Rook",(6,2))
 ("Rook",(7,2))
 ("Rook",(8,2))
 ("Rook",(2,2))
 ("Rook",(1,2))
 ("Rook",(3,1))
 ("King",(2,3))
 ("King",(2,4))
 ("King",(3,4))
 ("King",(4,3))
 ("King",(4,4))

In [ ]:
checks = (Int64,Int64)[]

checks = [checks, rookMoves((2,2),(1,1))]
println([checks,rookMoves((2,2),(1,1))])